# Inference with Gibbs Sampling

The file `data.npy` contains data sampled from a mixture of $K=4$ Gaussians. The standard deviations of the Gaussians are known and are identical, $\sigma_k = \sigma = 0.5$. 

Reconstruct the unknown parameters and latent variables,

$$
p_k , \mu_k, Z_i
$$

where $Z_i$ represents the assignment of each point to one of the mixture components.
To perform the inference, use Gibbs sampling:

* Initialize randomly $p_k , \mu_k, Z_i$ , then compute
$$ N_k = \sum_i \chi(Z_i = k)
\\
m_k = \sum_i \chi(Z_i = k) \cdot x_k 
$$

* Perform Gibbs Sampling iterations
    - Sample $\mu_k$ from a Normal with mean and standard deviations:
     $$ 
     \mu_k' = \bigg( \dfrac{\mu_0}{\sigma_0^2} + \dfrac{\mu_k}{\sigma^2} \bigg) \cdot \dfrac{1}{\bigg( \dfrac{1}{\sigma_0^2} + \dfrac{N_k}{\sigma^2} \bigg)} \quad , \quad \sigma_k' = \bigg( \dfrac{1}{\sigma_0^2} + \dfrac{N_k}{\sigma^2} \bigg)^{-1/2} 
     $$. 
     
     Here $\mu_0 = 0$ and $\sigma_0 = 1000$ are the Gaussian prior parameters. Do this for all $k$.


    - Sample the $p_k$ from a Dirichelet with parameters $\gamma_k' = \gamma_k + N_k$. Here $\gamma_k = 1$ is the Dirichelet prior parameter. Do this for all $k$.


    - Sample the $Z_i$ from a categorical distribution, $Prob(Z_i = k) = \dfrac{p_k \cdot e^{- \frac{(x_k - \mu_k)^2}{2 \sigma^2}}}{\sum_k p_k \cdot e^{- \frac{(x_k - \mu_k)^2}{2 \sigma^2}}}$
  
  
    If $Z_i$ is updated, update $N_k, m_k$ accordingly.

---

## The Gibbs Sampling Algorithm

Gibbs sampling is a Markov chain Monte Carlo (MCMC) technique for estimating the joint distribution of a set of variables. It can be used to perform Bayesian inference, which is a way of making statistical inferences about unknown quantities based on the principles of probability theory.

The basic idea behind Gibbs sampling is to iteratively sample from the full conditional distributions of each variable, given the current values of all the other variables. This process generates a sequence of samples from the joint distribution, which can be used to approximate the posterior distribution of the variables.

To perform Gibbs sampling, you need to specify the full conditional distributions for each variable, which can be derived from the joint distribution and the conditional independence structure of the variables. You then initialize the values of the variables and iterate through the following steps:

* For each variable, sample a new value from its full conditional distribution given the current values of the other variables.
* Repeat this process until the samples converge to a stationary distribution, which is an approximate representation of the joint distribution.



Gibbs sampling can be an effective way to perform Bayesian inference, particularly when the joint distribution is complex and it is difficult to sample directly from it. It is also relatively simple to implement and can be used to efficiently estimate posterior distributions in large-scale models.

---

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.special

import scipy.stats

In [2]:
#load the data from the file

dataset = np.loadtxt('data.txt')


In [3]:
# Visualizing the data

fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.hist(dataset, bins=np.arange(np.min(dataset) - 0.2 , np.max(dataset) + 0.2 , 0.2), 
        density=True, alpha = 0.5, edgecolor='black', zorder = 2)
ax.grid(zorder = 1)
ax.minorticks_on()
ax.set_title('Data PDF', fontsize = 15)
ax.set_ylabel('PDF', fontsize = 12)
plt.show()

In [4]:
K = 4

sigma = 0.5

In [5]:
# Define all the functions useful

def count_Nk(array):
    N_k_list = []
    
    for i in range(K):
        N_k_list.append(np.sum(array == i))
    
    return N_k_list

def count_mk(array, x):
    m_k_list = []
    
    for i in range(K):
        m_k_list.append(np.sum(array == i) * x[array == i])   #look to this, x has to be correlated to the assigned cluster
    
    return m_k_list

In [6]:
mu_0 = 0
sigma_0 = 1000

gamma_k = 1

In [18]:
#sample mu_k
def sample_mu_k(mu_k, Z_i):
    sampled_mu_k = np.zeros(K)
    
    for k in range(K):

        mu_k_prime = (mu_0/sigma_0**2 + mu_k[k]/sigma**2) / (1/sigma_0**2 + (count_Nk(Z_i)[k])/sigma**2)
        sigma_k_prime = (1/sigma_0**2 + (count_Nk(Z_i)[k])/sigma**2)**(-1/2)

        sampled_mu_k[k] = np.random.normal(mu_k_prime, sigma_k_prime)
        
    return sampled_mu_k
      
    
#sample p_k
def sample_p_k(Z_i):
    sampled_p_k = np.zeros(K)
    
    for k in range(K):
        gamma_prime_k = gamma_k + count_Nk(Z_i)[k]
        
        sampled_p_k[k] = np.random.dirichlet(gamma_prime_k)
    
    return sampled_p_k
    
    
#sample Z_i
def sample_Z_i(p_k, mu_k, Z_i, dataset):
    
    prob_Z_i_k = np.zeros(K)
    sampled_Z_i = np.zeros(len(dataset))
    
    for k in range(K):
        for i in range(len(dataset)):
            
            prob_Z_i_k[k] = (p_k[k] * np.exp(-(dataset[i] - mu_k[k])**2)) / ( np.sum(p_k[k] * np.exp(-(dataset[i] - mu_k[k])**2)) )
            
            sampled_Z_i[i] = np.random.choice(K, p = prob_Z_i_k)
            
    if sampled_Z_i != Z_i:
        count_Nk(sampled_Z_i)
        count_mk(sampled_Z_i, dataset)
                
    return sampled_Z_i
            

In [58]:
prob_Z_i_k = np.zeros(K)
sampled_Z_i = np.zeros(len(dataset))
    
   
for k in range(K):
    sum_norm = np.sum(p_k[k] * np.exp(-(dataset - mu_k[k])**2))
    
for i in range(len(dataset)):
    for k in range(K):
        prob_Z_i_k[k] = (p_k[k] * np.exp(-(dataset[i] - mu_k[k])**2)) / sum_norm 
        

sampled_Z_i = np.random.choice([0,1,2,3], p = [prob_Z_i_k[i] for i in range(K)] , size = len(dataset))

ValueError: probabilities do not sum to 1

In [56]:
   
for k in range(K):
        sum_norm = np.sum(p_k[k] * np.exp(-(dataset - mu_k[k])**2))

In [57]:
sum_norm

57.17001423253471

In [26]:
#Initialize the parameters

mu_k = np.random.uniform(np.min(dataset), np.max(dataset), K)

p_k = np.random.uniform(0, 1, K)

Z_i = np.random.randint(0, K, len(dataset))

# Gibbs Sampling

maxiter = 1000

    
 # List to store the values
mu_list = []
pk_list = []
Z_i_list = []

    
for i in range(maxiter):
        
    #proposals
    proposal_mu = sample_mu_k(mu_k, Z_i)
    proposal_pk = sample_mu_k(p_k, Z_i)
    proposal_zi = sample_Z_i(p_k, mu_k, Z_i, dataset)
    
    
    mu_list.append(proposal_mu)
    pk_list.append(proposal_pk)
    Z_i_list.append(proposal_zi)


        

ValueError: probabilities do not sum to 1

In [48]:
Z_i_try2 = np.random.choice([0,1,2,3], p = [0.5 , 0.2 , 0.2 , 0.1] , size = len(dataset))